In [2]:
#import libraries and modules
#from google.colab import files
import pandas as pd
#Supervised learning
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
##Deep learning libraries and APIs
import numpy as np
import pickle
import en_core_web_sm
from datetime import date, datetime, timedelta

today = date.today()


C:\Users\sreer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sreer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sreer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sreer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
model = pickle.load(open(r'C:\Users\sreer\Big data project\NLP_Headlines_analysis\logistic_model.sav', 'rb'))

In [4]:
df = pd.read_csv(r'C:\Users\sreer\Big data project\Headlines\headlines_{}.csv'.format(today))

In [5]:
df

Unnamed: 0                                          Headlines  \
0           0  Stock market holiday: BSE, NSE closed today on...   
1           1  US stocks dip, bond yields climb on tightening...   

                     Date  
0  Apr 15, 2022, 09:18 AM  
1  Apr 15, 2022, 08:48 AM

In [6]:
df1 = df.drop(['Unnamed: 0'], axis=1)

In [7]:
columns=["Date","Headlines"]
df1 = df1.reindex(columns=columns)
df1

Date                                          Headlines
0  Apr 15, 2022, 09:18 AM  Stock market holiday: BSE, NSE closed today on...
1  Apr 15, 2022, 08:48 AM  US stocks dip, bond yields climb on tightening...

In [8]:
df = df.drop(['Date','Unnamed: 0'], axis=1)

In [9]:
df

Headlines
0  Stock market holiday: BSE, NSE closed today on...
1  US stocks dip, bond yields climb on tightening...

In [10]:
new_data = df['Headlines'].astype(str).values.tolist()
new_data

['Stock market holiday: BSE, NSE closed today on account of Good Friday',
 'US stocks dip, bond yields climb on tightening concerns']

In [11]:
data = pd.read_csv(r'C:\Users\sreer\Big data project\NLP_Headlines_analysis\data_file.csv')
data

Unnamed: 0                                               text
0              0  evacuate live ally nato 3 bucharest indian ukr...
1              7  flight evacuate nifty indian podcast news ukra...
2             12  nifty50 16,650 steel surge 6 pt sensex stock 1...
3             14  remain focus previous nifty50 crisis bell ukra...
4             15              dlf 25 indus stock tower key move feb
...          ...                                                ...
2060        4981  modi meet nifty podcast news kamala sensex har...
2061        4983  robust last defence paras ipo 304 time far sub...
2062        4986  leader pandemic fresh post rally high reliance...
2063        4992  finserv sbi stock real estate bajaj key move 2...
2064        4996  economy indian positive improvement see struct...

[2065 rows x 2 columns]

In [12]:
tf = TfidfVectorizer()
tfdf = tf.fit_transform(data['text'])
vect = pd.DataFrame(tf.transform(new_data).toarray())
new_data = pd.DataFrame(vect)
logistic_prediction = model.predict(new_data)
print(logistic_prediction)

[1 1]


In [13]:
Sentiment_result = pd.DataFrame(logistic_prediction,columns=['Sentiment_type'])
Sentiment_result

Sentiment_type
0               1
1               1

In [14]:
df1.reset_index(drop=True, inplace=True)
Sentiment_result.reset_index(drop=True, inplace=True)

GCP_upload = pd.concat( [df1, Sentiment_result], axis=1) 
GCP_upload

Date                                          Headlines  \
0  Apr 15, 2022, 09:18 AM  Stock market holiday: BSE, NSE closed today on...   
1  Apr 15, 2022, 08:48 AM  US stocks dip, bond yields climb on tightening...   

   Sentiment_type  
0               1  
1               1

In [15]:
GCP_upload['Sentiment_type'] = GCP_upload['Sentiment_type'].map({1: 'Positive', 0: 'Negative',-1:'Neutral'})
GCP_upload

Date                                          Headlines  \
0  Apr 15, 2022, 09:18 AM  Stock market holiday: BSE, NSE closed today on...   
1  Apr 15, 2022, 08:48 AM  US stocks dip, bond yields climb on tightening...   

  Sentiment_type  
0       Positive  
1       Positive

In [ ]:
positive_sentiment = sum([1 for itr in logistic_prediction if  itr==1])
negative_sentiment = len(logistic_prediction) - positive_sentiment
result = positive_sentiment - negative_sentiment
print(result)
resultset = {"total_headlines":len(logistic_prediction),"positive_sentiment":positive_sentiment,"negative_sentiment":negative_sentiment,"Sentiment_type_of_the_day":"Neutral"}
if result > 0:
    resultset["Sentiment_type_of_the_day"]="Positive"
elif result == 0:
    resultset["Sentiment_type_of_the_day"]="Neutral"
else:
    resultset["Sentiment_type_of_the_day"]="Negative"
    

In [ ]:
import json

with open(r'C:\Users\sreer\Big data project\Stock_results\Sentiment_result_{}.json'.format(today), 'w') as fp:
    json.dump(resultset, fp)

In [15]:
import io
from io import BytesIO
from google.cloud import storage

In [16]:
storage_client=storage.Client.from_service_account_json(r"C:\Users\sreer\Big data project\NLP_Headlines_analysis\stock-mail-347423-32544bc425c9.json")
bucket = storage_client.get_bucket("stock_mail")
bucket.blob(f'Headlines_db/Headlines_{today}.csv').upload_from_string(GCP_upload.to_csv(), 'text/csv')